In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.contrib import learn
from sklearn.metrics import mean_squared_error
from lstm import lstm_model, prepare_and_split_data
from tensorflow.python.framework import dtypes
from tensorflow.contrib import learn

In [ ]:
import tflearn

In [ ]:
# LOG_DIR = './ops_logs/all_points_model'
# TIMESTEPS = 5
# RNN_LAYERS = [{'num_units': 40}, {'num_units': 20}]
# DENSE_LAYERS = [10]
# TRAINING_STEPS = 10000
# PRINT_STEPS = TRAINING_STEPS / 10
# BATCH_SIZE = 100

LOG_DIR = './ops_logs'
TIMESTEPS = 10
RNN_LAYERS = [{'num_units': TIMESTEPS}]
DENSE_LAYERS = [10, 10]
TRAINING_STEPS = 100000
BATCH_SIZE = 100
PRINT_STEPS = TRAINING_STEPS / 100

In [ ]:
X, y = prepare_and_split_data(pd.read_csv('mydata.csv'), TIMESTEPS)

In [ ]:
regressor = learn.Estimator(model_fn=lstm_model(TIMESTEPS, RNN_LAYERS, DENSE_LAYERS), model_dir=LOG_DIR)

In [ ]:
# create a lstm instance and validation monitor
validation_monitor = learn.monitors.ValidationMonitor(X['val'], y['val'],every_n_steps=PRINT_STEPS,
                                                                                          early_stopping_rounds=1000)
regressor.fit(X['train'], y['train'], monitors=[validation_monitor], batch_size=BATCH_SIZE, steps=TRAINING_STEPS)

In [ ]:
predicted = regressor.predict(X['test'])
corrected_predictions = np.zeros(predicted.shape)

for i, time in enumerate(predicted):
    for j, point in enumerate(time):
        if point <=1.05:
            corrected_predictions[i,j] = 1.0
        else:
            corrected_predictions[i,j] = point

In [ ]:
new_score = mean_squared_error(corrected_predictions, np.nan_to_num(y['test']))
print ("NEW MSE: %f" % new_score)

# plotting graphs below here

In [ ]:
#[0, 1, 2, 3, 14, 15, 16, 12, 13
plt.figure(figsize=(20,40))

plt.subplot(911)
plt.plot(corrected_predictions[:,0][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,0][600:1000],label='true 0')
plt.legend()

plt.subplot(912)
plt.plot(corrected_predictions[:,1][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,1][600:1000],label='true 1')
plt.legend()

plt.subplot(913)
plt.plot(corrected_predictions[:,2][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,2][600:1000],label='true 2')
plt.legend()

plt.subplot(914)
plt.plot(corrected_predictions[:,3][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,3][600:1000],label='true 3')
plt.legend()

plt.subplot(915)
plt.plot(corrected_predictions[:,4][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,4][600:1000],label='true 14')
plt.legend()

plt.subplot(916)
plt.plot(corrected_predictions[:,5][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,5][600:1000],label='true 15')
plt.legend()

plt.subplot(917)
plt.plot(corrected_predictions[:,6][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,6][600:1000],label='true 16')
plt.legend()

plt.subplot(918)
plt.plot(corrected_predictions[:,7][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,7][600:1000],label='true 12')
plt.legend()

plt.subplot(919)
plt.plot(corrected_predictions[:,8][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,8][600:1000],label='true 13')
plt.legend()

plt.show()

In [ ]:
# , 24, 25, 26, 27, 28, 17, 29]
plt.figure(figsize=(20,30))

plt.subplot(711)
plt.plot(corrected_predictions[:,9][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,9][600:1000],label='true 24')
plt.legend()

plt.subplot(712)
plt.plot(corrected_predictions[:,10][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,10][600:1000],label='true 25')
plt.legend()

plt.subplot(713)
plt.plot(corrected_predictions[:,11][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,11][600:1000],label='true 26')
plt.legend()

plt.subplot(714)
plt.plot(corrected_predictions[:,12][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,12][600:1000],label='true 27')
plt.legend()

plt.subplot(715)
plt.plot(corrected_predictions[:,13][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,13][600:1000],label='true 28')
plt.legend()

plt.subplot(716)
plt.plot(corrected_predictions[:,14][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,14][600:1000],label='true 17')
plt.legend()

plt.subplot(717)
plt.plot(corrected_predictions[:,15][600:1000],label='5 min forecast')
plt.plot(np.nan_to_num(y['test'])[:,15][600:1000],label='true 29')
plt.legend()

plt.show()